# Search Augmented

### by Query Generation and Embeddings Reranking

Searching for relevant information can sometimes be like looking for a needle in a haystack, but don’t despair, GPTs can actually do a lot of this work for us. In this guide we explore a way to augment existing search systems with various AI techniques, helping us sift through the noise.

There are two prominent approaches to using language models for information retrieval:

1. **Mimicking Human Browsing:** The [model triggers a search](https://openai.com/blog/chatgpt-plugins#browsing), evaluates the results, and modifies the search query if necessary. It can also follow up on specific search results to form a chain of thought, much like a human user would do.
2. **Retrieval with Embeddings:** Calculating [embeddings](https://platform.openai.com/docs/guides/embeddings) for your content, and then using a metric like cosine distance between the user query and the embedded data to sort and [retrieve information](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb). This technique is [used heavily](https://blog.google/products/search/search-language-understanding-bert/) by search engines like Google.

These approaches are both promising, but each has their shortcomings: the first one can be slow due to its iterative nature and the second one requires embedding your entire knowledge base in advance, continuously embedding new content and maintaining a vector database.

By combining these approaches, and drawing inspiration from [re-ranking](https://www.sbert.net/examples/applications/retrieve_rerank/README.html) methods, we identify an approach that sits in the middle. **This approach can be implemented on top of any existing search system, like the Slack search API, or an internal ElasticSearch instance with private data**. Here’s how it works:

![search_augmented_by_query_generation_and_embeddings_reranking.png](../images/search_augmentation_embeddings.png)

**Step 1: Search**

1.  User asks a question
2.  Model generates a list of potential queries
3.  Search queries are executed in parallel

**Step 2: Re-rank**

1.  Embeddings for each result are used to calculate semantic similarity to a generated hypothetical ideal answer to the user question.
2.  Results are ranked and filtered based on this similarity metric.

**Step 3: Answer**

1.  Given the top search results, the model generates an answer to the user’s question, including references and links.

This hybrid approach offers relatively low latency and can be integrated into any existing search endpoint, without requiring the upkeep of a vector database. Let's dive into it! We will use the [News API](https://newsapi.org/) as an example domain to search over.

## Setup

In addition to your `OPENAI_API_KEY`, you'll have to include a `NEWS_API_KEY` in your environment. You can get the API key [here](https://newsapi.org/).


In [1]:
%env NEWS_API_KEY = YOUR_API_KEY


In [2]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython import display  # for pretty printing
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import openai
import os  # for loading environment variables
import requests  # for making the API requests
from tqdm import tqdm  # for printing progress bars

# Load environment variables
news_api_key = os.getenv("NEWS_API_KEY")


# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=1,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input: list[str]) -> list[list[str]]:
    response = openai.Embedding.create(
        model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]


## 1. Search

It all starts with a user question.


In [3]:
# User asks a question
USER_QUESTION = "Who won the NBA championship? And who was the MVP? Tell me a bit about the last game."

Now, in order to be as exhaustive as possible, we use the model to generate a list of diverse queries based on this question.


In [4]:
QUERIES_INPUT = f"""
Generate an array of search queries that are relevant to this question. 
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms. 
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2']. 
Be creative. The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

# Lets include the original question as well for good measure
queries.append(USER_QUESTION)

queries


['NBA championship winner',
 'NBA finals MVP',
 'last NBA championship game',
 'recent NBA finals results',
 'NBA finals champions',
 'NBA finals MVP and winner',
 'latest NBA championship game details',
 'NBA championship winning team',
 'most recent NBA finals MVP',
 'last NBA finals game summary',
 'latest NBA finals champion and MVP',
 'Who won the NBA championship? And who was the MVP? Tell me a bit about the last game.']

The queries look good, let's run the searches.


In [5]:
def search_news(query: str):
    # get date 1 week ago
    one_week_ago = (date.today() - timedelta(weeks=1)).strftime("%Y-%m-%d")

    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": 50,
            "sortBy": "relevancy",
            "from": one_week_ago,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    result = search_news(query)
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

100%|██████████| 12/12 [00:04<00:00,  2.69it/s]


In [6]:
print("Total number of articles:", len(articles))
print("Top 5 articles of query 1:", "\n")

for article in articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print()

Total number of articles: 356
Top 5 articles of query 1: 

Title: Nascar takes on Le Mans as LeBron James gets centenary race under way
Description: <ul><li>Nascar has presence at iconic race for first time since 1976</li><li>NBA superstar LeBron James waves flag as honorary starter</li></ul>The crowd chanted “U-S-A! U-S-A!” as Nascar driver lineup for the 24 Hours of Le Mans passed through the city cente…
Content: The crowd chanted U-S-A! U-S-A! as Nascar driver lineup for the 24 Hours of Le Mans passed through t...

Title: Futura and Michelob ULTRA Toast to the NBA Finals With Abstract Artwork Crafted From the Brand’s 2023 Limited-Edition Championship Bottles
Description: The sun is out to play, and so is Michelob ULTRA. With the 2022-2023 NBA Finals underway, the beermaker is back with its celebratory NBA Champ Bottles. This year, the self-proclaimed MVP of joy is dropping a limited-edition bottle made in collaboration with a…
Content: The sun is out to play, and so is Michelob ULTR

As we can see, oftentimes, the search queries will return a large number of results, many of which are not relevant to the original question asked by the user. In order to improve the quality of the final answer, we use embeddings to re-rank and filter the results.

# 2. Re-rank

Drawing inspiration from [HyDE (Gao et. al.)](https://arxiv.org/abs/2212.10496), we first generate a hypothetical ideal answer to rerank our compare our results against. This ensures that we’re prioritizing results that might provide good answers, rather than those that are similar to our question. Here’s the prompt we use to generate our hypothetical answer.


In [7]:
HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer which will be used to rank search results. 
Pretend you have all the information you need to answer, but don't use any actual facts. Instead, use placeholders
like NAME did something, or NAME said something at PLACE. 

User question: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer

'Team NAME won the NBA championship, and PLAYER_NAME was awarded the MVP title. In the last game, NAME displayed an outstanding performance with X points, Y rebounds, and Z assists, leading their team to a thrilling victory at PLACE.'

Now, let's generate embeddings for the search results and the hypothetical answer. We then calculate the cosine distance between these embeddings, giving us a semantic similarity metric. Note that we can simply calculate the dot product in lieu of doing a full cosine similarity calculation since the OpenAI embeddings are returned normalized in our API.


In [8]:
hypothetical_answer_embedding = embeddings(hypothetical_answer)
article_embeddings = embeddings(
    [
        f"{article['title']} {article['description']} {article['content'][0:100]}"
        for article in articles
    ]
)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding)[0])

cosine_similarities[0:10]

[0.779471308147333,
 0.7800588417397301,
 0.7900892607044301,
 0.7704583513281005,
 0.7841046909560899,
 0.8246545759453099,
 0.8127694680991286,
 0.8235724294003601,
 0.7978980332478777,
 0.8273641985639677]

Finally, we use these similarity scores to sort and filter the results.


In [9]:
scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

# Print top 5 articles
print("Top 5 articles:", "\n")

for article, score in sorted_articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()

Top 5 articles: 

Title: Barack Obama and Magic Johnson lead Nuggets tributes after first-ever NBA title win
Description: Straight after the Nuggets' Game 5 win, several standout names and brands within the world of sports and elsewhere paid tribute to Nikola Jokic and his teammates for their historic season.
Content: The Denver Nuggetsclinched their first ever NBA championship on Monday night, following it's Game 5 ...
Score: 0.8440720976260312

Title: Barack Obama and Magic Johnson lead Nuggets tributes after first-ever NBA title win
Description: Straight after the Nuggets' Game 5 win, several standout names and brands within the world of sports and elsewhere paid tribute to Nikola Jokic and his teammates for their historic season.
Content: The Denver Nuggetsclinched their first ever NBA championship on Monday night, following it's Game 5 ...
Score: 0.8440720976260312

Title: Nikola Jokic wins NBA Finals MVP, leading Denver Nuggets to first championship
Description: In the 2023 NBA F

Awesome! These results look a lot more relevant to our original query. Now, let's use the top 20 results to generate a final answer.

## 3. Answer


In [11]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. Include references to the search results as markdown links.
"""

completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=1,
    stream=True,
)

text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))


The Denver Nuggets won their first-ever NBA championship, and Nikola Jokic was named the NBA Finals MVP. In the last game, Game 5 of the 2023 NBA Finals, Jokic became the first player ever with a 30-point, 20-rebound triple-double in a Finals game [^1^] [^3^]. The Nuggets' historic season was celebrated and recognized by standout names and brands within the world of sports and elsewhere, including tributes from Barack Obama and Magic Johnson [^1^] [^2^].

[^1^]: [Denver Post](https://www.denverpost.com/2023/06/12/nikola-jokic-nba-finals-mvp-denver-nuggets/)
[^2^]: [Daily Mail](https://www.dailymail.co.uk/sport/nba/article-12189843/Barack-Obama-Magic-Johnson-lead-Nuggets-tributes-NBA-title-win.html)
[^3^]: [USA Today](https://www.usatoday.com/story/sports/nba/2023/06/12/nikola-jokic-is-nba-finals-mvp-after-nuggets-win-first-championship/70314309007/)